In [1]:
import numpy as np
import matplotlib.pyplot as plt

X_train: きのこ
cap color 傘の色, stalk shape 形, solitary 単一か

y_train: 食用1か否か0

In [2]:
X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])

In [3]:
print(f"x_train.shape:{X_train.shape}")
print(f"y_train.shape:{y_train.shape}")

x_train.shape:(10, 3)
y_train.shape:(10,)


entropy  
$$H(p_1) = -p_1 \text{log}_2(p_1) - (1- p_1) \text{log}_2(1- p_1)$$

In [4]:
def compute_entropy(y):
    entropy = 0.0
    # code
    if len(y != 0):
        p1 = len(y[y == 1]) / len(y)
        if p1 == 1 or p1 == 0:
            entropy = 0
        else:
            entropy = -p1 * np.log2(p1) - (1 - p1) * np.log2(1 - p1)
    # code
    return entropy

In [5]:
print(compute_entropy(y_train))

1.0


In [6]:
def split_dataset(x, node_indices, feature):
    left_indices = [] # Indices with feature value == 1
    right_indices = [] # Indices with feature value == 0
    # code
    for i in node_indices:
        if x[i][feature] == 1:
            left_indices.append(i)
        else:
            right_indices.append(i)
    # code
    return left_indices, right_indices

Information Gain
$$\text{Information Gain} = H(p_1^\text{node})- (w^{\text{left}}H(p_1^\text{left}) + w^{\text{right}}H(p_1^\text{right}))$$

In [7]:
def compute_information_gain(X, y, node_indices, feature):
    left_indices, right_indices = split_dataset(X, node_indices, feature)
    X_node, y_node = X[node_indices], y[node_indices]
    X_left, y_left = X[left_indices], y[left_indices]
    X_right, y_right = X[right_indices], y[right_indices]
    information_gain = -1
    # code
    node_entropy = compute_entropy(y_node)
    left_entropy = compute_entropy(y_left)
    right_entropy = compute_entropy(y_right)
    w_left = len(X_left) / len(X_node)
    w_right = len(X_right) / len(X_node)
    information_gain = node_entropy - (w_left * left_entropy + w_right * right_entropy)
    # code
    return information_gain

information_gainが一番大きいデータの分け方が分けるべき特徴を表す

In [8]:
def get_best_split(X, y, node_indices):
    num_features = X.shape[1]
    best_feature = -1 # invalid
    # code
    max_information_gain = 0 # invalid
    for feature in range(num_features):
        gain = compute_information_gain(X, y, node_indices, feature)
        if gain == 1:
            return -1
        else:
            if gain > max_information_gain:
                max_information_gain = gain
                best_feature = feature
    # code
    return best_feature

In [9]:
tree = []
def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth):
    # Maximum depth reached - stop splitting
    if current_depth == max_depth:
        formatting = " "*current_depth + "-"*current_depth
        print(formatting, "%s leaf node with indices" % branch_name, node_indices)
        return
   
    # Otherwise, get best split and split the data
    # Get the best feature and threshold at this node
    best_feature = get_best_split(X, y, node_indices) 
    
    formatting = "-"*current_depth
    print("%s Depth %d, %s: Split on feature: %d" % (formatting, current_depth, branch_name, best_feature))
    
    # Split the dataset at the best feature
    left_indices, right_indices = split_dataset(X, node_indices, best_feature)
    tree.append((left_indices, right_indices, best_feature))
    
    # continue splitting the left and the right child. Increment current depth
    build_tree_recursive(X, y, left_indices, "Left", max_depth, current_depth+1)
    build_tree_recursive(X, y, right_indices, "Right", max_depth, current_depth+1)

In [10]:
root_indices = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [11]:
build_tree_recursive(X_train, y_train, root_indices, "Root", max_depth=2, current_depth=0)

 Depth 0, Root: Split on feature: 2
- Depth 1, Left: Split on feature: 0
  -- Left leaf node with indices [0, 1, 4, 7]
  -- Right leaf node with indices [5]
- Depth 1, Right: Split on feature: 1
  -- Left leaf node with indices [8]
  -- Right leaf node with indices [2, 3, 6, 9]
